In [1]:
import urllib, demjson, os, json, requests, time, random
import pandas as pd 
import numpy as np

In [100]:
datelist = list(range(2009,2018))
best_sellers = pd.read_csv('best_sellers.csv')

In [109]:
adjectives = '''
unusual
torpid
spiteful
scattered
daffy
neighborly
unsuitable
intelligent
modern
bright
aromatic
frightening
fabulous
garrulous
crooked
comfortable
relieved
uncovered
nutritious
bored
tart
historical
scientific
fantastic
absorbing
unbecoming
aquatic
shaggy
puny
callous
minor
unwritten
petite
energetic
groovy
ruddy
truculent
placid
untidy
pleasant
funny
acrid
incandescent
rapid
royal
living
unable
guiltless
deceive
wrestle
perform
harm
plan
invite
cheer
compare
bomb
jail
end
consider
live
tow
shelter
bump
bubble
fear
stretch
trade
tip
found
enter
wash
coil
curve
blink
thaw
unlock
stay
hook
bore
consist
ban
examine
taste
multiply
report
flower
join
tempt
dust
separate
follow
bare
stuff
mine
complain
decide
wriggle
wait
gather
examine
seal
possess
moan
marry
coil
belong
measure
squeak
carve
whirl
enter
murder
scratch
precede
annoy
pop
pretend
crush
look
drown
fasten
order
protect
rush
smoke
thank
twist
float
snore
lock
chew
invent
bore
trust
excuse
hope
follow
license
alert
correct
attach
joke
rescue
wriggle
fry
paddle
deceive
hanging
innate
unusual
torpid
spiteful
scattered
daffy
neighborly
unsuitable
intelligent
modern
bright
aromatic
frightening
fabulous
garrulous
crooked
comfortable
relieved
uncovered
nutritious
bored
tart
historical
scientific
fantastic
absorbing
unbecoming
aquatic
shaggy
puny
callous
minor
unwritten
petite
energetic
groovy
ruddy
truculent
placid
untidy
pleasant
funny
acrid
incandescent
rapid
royal
living
unable
guiltless'''

In [110]:
adjlist = adjectives.split('\n')

In [117]:
json_non = []
for word in adjlist:
    for i in range(100):
        url = f'''https://www.googleapis.com/books/v1/volumes?q={word}&maxResults=1&startIndex={i}'''
        response = urllib.request.urlopen(url)
        temp_data = json.loads(response.read())
        json_non.append(temp_data)
        time.sleep(random.uniform(.5, 1))

HTTPError: HTTP Error 400: Bad Request

In [91]:
#find the url in the json file
def gettingUrl(file):
    try:
        current = json.loads(json.dumps(file))['items']
        current = json.loads(json.dumps(current[0]))['volumeInfo']
        title = json.loads(json.dumps(current))['title']
        author = json.loads(json.dumps(current))['authors'][0]
        publishedDate = json.loads(json.dumps(current))['publishedDate']
        publishedYear = int(publishedDate[0:4])
        current = json.loads(json.dumps(current))['imageLinks']
        url = json.loads(json.dumps(current))['smallThumbnail']
        if ((title in best_sellers['title']) or (publishedYear not in datelist)):
            return 'fail', '', ''
        else:
            return url, title, author
    except KeyError:
        return 'fail', '', ''
    except ValueError:
        return 'fail', '', ''

In [107]:
len(json_non)

160

In [108]:
# non_best_sellers = pd.DataFrame(columns=['title','author','image'])
for idx, jsonFile in enumerate(json_non):
    curr_url, title, author = (gettingUrl(jsonFile))
    if (curr_url == 'fail'):
        continue
    img_data = requests.get(curr_url).content
    title = title.replace('/',' ')
    with open(f'non_best_sellers_images/{title}.jpg', 'wb') as handler:
        handler.write(img_data)
    temp_df = pd.DataFrame([[title,author,f'{title}.jpg']],columns=['title','author','image'])
    non_best_sellers = non_best_sellers.append(temp_df)

In [113]:
non_best_sellers = non_best_sellers.drop_duplicates(subset=['title'])
non_best_sellers['best'] = 0
non_best_sellers.to_csv('non_best_sellers.csv')